In [ ]:
import v20
import configparser

config = configparser.ConfigParser()
config.read('config.ini')

API_KEY = config['OANDA2']['API_KEY'] 
ACCOUNT_ID = config['OANDA2']['ACCOUNT_ID']
HOSTNAME   = "api-fxpractice.oanda.com"
STREAMHOST = "stream-fxpractice.oanda.com"
key = f'Bearer {API_KEY}'

# contexts
ctx = v20.Context(hostname=HOSTNAME, token=key)
ctx.set_header(key='Authorization', value=key)

In [ ]:
ac = ctx.account.summary(ACCOUNT_ID).get('account')

In [ ]:
ti = ['EUR_USD', 'EUR_CAD', 'EUR_NZD', 'EUR_CHF', 'EUR_JPY', 'EUR_AUD', 'EUR_GBP', 'GBP_USD', 'GBP_CAD', 'GBP_JPY', 'GBP_AUD',
      'AUD_USD', 'AUD_CAD', 'AUD_NZD', 'AUD_JPY', 'NZD_USD', 'NZD_JPY', 'USD_CHF', 'USD_CAD', 'USD_JPY']
ti_csv = ','.join(ti)

In [ ]:
insts = ctx.account.instruments(ACCOUNT_ID, instruments=ti_csv).get('instruments')

In [ ]:
print(insts[0])

In [ ]:
insts[0].displayPrecision

In [ ]:
inst = insts[0]
u = 105
units = f'{u:.{inst.tradeUnitsPrecision}f}'

gp_sl = 15 * pow(10, inst.pipLocation)
gp_ts = 30 * pow(10, inst.pipLocation)

sl_on_fill = dict(
    timeInForce='GTC', 
    # price=f'{sl_price:.{insts.displayPrecision}f}',
    distance=f'{gp_sl:.{inst.displayPrecision}f}')

#tp_on_fill = dict(
#    timeInForce='GTC', 
#    price=f'{tp_price:.{inst.displayPrecision}f}')

ts_on_fill = dict(
    timeInForce='GTC', 
    distance=f'{gp_ts:.{inst.displayPrecision}f}')

order = dict(
    type='MARKET',
    instrument=inst.name,
    units=units,
    #takeProfitOnFill=tp_on_fill,
    stopLossOnFill=sl_on_fill,
    trailingStopLossOnFill=ts_on_fill
)

print(order)
response = ctx.order.market(ACCOUNT_ID, **order)
print(response)

for b in response.body:
    print(response.get(b))
    print('')

In [ ]:
olp=ctx.order.list_pending(ACCOUNT_ID)